In [34]:
# import the OpenCV package
import cv2
import os
import random
import numpy as np
import smartcrop
from PIL import Image
from pprint import pprint
import shutil

In [35]:
sc = smartcrop.SmartCrop()

def translate_image(img, x, y):
    num_rows, num_cols = img.shape[:2]
    translation_matrix = np.float32([[1, 0, x], [0, 1, y]])
    img_translation = cv2.warpAffine(
        img, translation_matrix, (num_cols, num_rows))
    return img_translation[max(y, 0):num_rows-abs(y), max(x, 0):num_cols-abs(x)]

def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

def add_noise_saltpepper(image, amount):
    row, col, ch = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    out[coords] = 1
    # Pepper mode
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    out[coords] = 0
    return out

def add_noise_gaussian(image, mean: float, var: float):
    row, col, ch = image.shape
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    return noisy

In [36]:
dirname = '/Volumes/TriveStorage/code/trive-image-recognition/complete_manual/directional'

def mkdir_if_missing(p: str):
    if os.path.exists(p):
        # shutil.rmtree(p, ignore_errors=True)
        return
    os.mkdir(p)

# vehicle_classes = [x for x in os.listdir(dirname) if '.DS_Store' not in x]
vehicle_classes = ['기아_카니발_올 뉴 카니발(14~18년)',
 '기아_쏘렌토_올 뉴 쏘렌토(14~17년)',
 '기아_카니발_그랜드 카니발(05~10년)',
 '기아_카니발_카니발 R(10~14년)',
 '기아_K5_더 뉴 K5(13~15년)',
 '기아_K7_올 뉴 K7(16년~현재)',
 '기아_레이_레이(11~17년)',
 '기아_모닝_올 뉴 모닝(11~15년)',
 '기아_쏘렌토_쏘렌토 R(09~12년)',
 '기아_K7_더 뉴 K7(12~16년)',
 '기아_K5_K5 2세대(15~18년)',
 '기아_모하비_모하비(07~16년)',
 '기아_스포티지_스포티지 R(10~13년)',
 '기아_스포티지_더 뉴 스포티지 R(13~15년)',
 '기아_스포티지_New 스포티지(04~10년)',
 '르노삼성_SM6_SM6 (16년~현재)',
 '기아_스포티지_스포티지 4세대(15~18년)',
 '기아_모닝_더 뉴 모닝(15~17년)',
 '기아_모닝_올 뉴 모닝 (JA)(17년~현재)',
 '기아_쏘렌토_뉴 쏘렌토 R(12~14년)',
 '기아_모하비_더 뉴 모하비(16년~현재)',
 '기아_K3_K3(12~15년)',
 '기아_모닝_뉴모닝(08~11년)',
 '지프_랭글러_랭글러 (JK)(07~18년)',
 '제네시스_EQ900_EQ900(15~18년)',
 '기아_K7_K7(09~11년)',
 '기아_쏘렌토_더 뉴 쏘렌토(17년~현재)',
 '기아_쏘렌토_쏘렌토(02~06년)',
 '르노삼성_SM5_뉴SM5(신형) (10~12년)',
 '제네시스_G80_G80(16년~현재)',
 '르노삼성_QM3_ QM3 (13~17년)',
 '지프_체로키_그랜드 체로키(93년~현재)',
 '기아_오피러스_뉴오피러스(06~09년)',
 '기아_쏘렌토_뉴쏘렌토(06~09년)',
 '기아_K3_더 뉴 K3(15~18년)',
 '르노삼성_QM6_ QM6 (16년~현재)',
 '기아_K9_더 뉴 K9(14~18년)',
 '기아_오피러스_오피러스 프리미엄(09~11년)',
 '기아_니로_니로(16~19년)',
 '기아_K9_K9(12~14년)',
 '르노삼성_SM5_뉴SM5 플래티넘 (12~15년)',
 '기아_K7_더 프레스티지 K7(11~12년)',
 '르노삼성_SM5_SM5 뉴 임프레션 (07~10년)',
 '기아_K5_K5 하이브리드(11~15년)',
 '기아_쏘울_쏘울(08~13년)',
 '기아_카니발_더 뉴 카니발(18년~현재)',
 '르노삼성_SM5_SM5 노바(15년~현재)',
 '기아_프라이드_프라이드(신형)(05~11년)',
 '현대_싼타페_싼타페 더 프라임(15~18년)',
 '기아_카렌스_뉴카렌스(06~13년)',
 '기아_프라이드_올 뉴 프라이드(11~15년)',
 '르노삼성_SM7_ SM7 New Art (08~11년)',
 '르노삼성_SM3_SM3 네오(14년~현재)',
 '기아_K3_올 뉴 K3(18년~현재)',
 '현대_그랜저_그랜저 IG(16년~현재)',
 '기아_스팅어_스팅어(17년~현재)',
 '르노삼성_SM3_뉴SM3 (09~14년)',
 '벤츠_E-클래스_ E-클래스 W213 (16년~현재)',
 '기아_봉고III 미니버스_봉고III 미니버스(04~05년)',
 '기아_카니발_뉴카니발(06~10년)',
 '인피니티_Q50_Q50 (14년~현재)',
 '현대_싼타페_싼타페 TM(18년~현재)',
 '기아_포르테_포르테(08~13년)',
 '쉐보레(GM대우)_스파크_더 넥스트 스파크(15~18년)',
 '기아_스토닉_스토닉(17년~현재)',
 '벤츠_E-클래스_ E-클래스 W212 (09~17년)',
 '현대_쏘나타_쏘나타 뉴 라이즈(17~19년)',
 '르노삼성_QM5_ QM5 (07~11년)',
 '쉐보레(GM대우)_스파크_스파크(11~15년)',
 '기아_K9_더 K9(18년~현재)',
 '르노삼성_SM7_SM7 노바 (14년~현재)',
 '현대_제네시스_제네시스 DH(13~16년)',
 '현대_쏘나타_LF 쏘나타(14~17년)',
 '기아_K7_올 뉴 K7 하이브리드(16년~현재)',
 '르노삼성_SM5_뉴SM5 (05~07년)',
 '현대_스타렉스_더 뉴 그랜드 스타렉스(17년~현재)',
 '현대_에쿠스_에쿠스(신형)(09~15년)',
 '기아_쏘울_올 뉴 쏘울(13~16년)',
 '벤츠_C-클래스_ C-클래스 W205 (14년~현재)',
 '벤츠_S-클래스_ S-클래스 W221 (05~13년)',
 '르노삼성_SM7_ All NewSM7 (11~14년)',
 '기아_K5_더 뉴 K5 2세대(18년~현재)',
 '포르쉐_카이엔_뉴 카이엔(10~17년)',
 '기아_스포티지_스포티지 더 볼드(18년~현재)',
 '기아_포르테_포르테 쿱(09~13년)',
 '제네시스_G70_G70(17년~현재)',
 '기아_레이_더 뉴 레이(17년~현재)',
 '현대_그랜저_그랜저 HG(11~16년)',
 '현대_싼타페_싼타페 DM(12~15년)',
 '쌍용_렉스턴_G4 렉스턴 (17년~현재)',
 '현대_맥스크루즈_더 뉴 맥스크루즈(15~19년)',
 '기아_프라이드_더 뉴 프라이드(15~17년)',
 '기아_카렌스_올 뉴 카렌스(13~16년)',
 '지프_체로키_체로키(KL)(14년~현재)',
 '쉐보레(GM대우)_올란도_올란도(11~18년)',
 '르노삼성_QM5_ 뉴QM5 (11~14년)',
 '르노삼성_QM5_ QM5 네오 (14~16년)',
 '현대_코나_코나(17년~현재)',
 '현대_아반떼_아반떼 AD(15년~18년)',
 '쉐보레(GM대우)_말리부_올 뉴 말리부(16~18년)']

random.shuffle(vehicle_classes)

directions = ['back', 'front', 'mix', 'side']
for vehicle_class in vehicle_classes:
    print(vehicle_class)
    vehicle_path = os.path.join(dirname, vehicle_class)

    paths_hflip = [os.path.join(vehicle_path, f'{x}_hflip') for x in directions]
    for p in paths_hflip:
        mkdir_if_missing(p)

    paths_vflip = [os.path.join(vehicle_path, f'{x}_vflip') for x in directions]
    for p in paths_vflip:
        mkdir_if_missing(p)

    paths_rot90 = [os.path.join(vehicle_path, f'{x}_rot90') for x in directions]
    for p in paths_rot90:
        mkdir_if_missing(p)

    paths_rot270 = [os.path.join(vehicle_path, f'{x}_rot270') for x in directions]
    for p in paths_rot270:
        mkdir_if_missing(p)

    paths_smartcrop = [os.path.join(vehicle_path, f'{x}_smartcrop') for x in directions]
    for p in paths_smartcrop:
        mkdir_if_missing(p)
                 
    paths_saltpepper0d01 = [os.path.join(vehicle_path, f'{x}_saltpepper0d01') for x in directions]
    for p in paths_saltpepper0d01:
        mkdir_if_missing(p)
                 
    paths_saltpepper0d02 = [os.path.join(vehicle_path, f'{x}_saltpepper0d02') for x in directions]
    for p in paths_saltpepper0d02:
        mkdir_if_missing(p)
        
    paths_saltpepper0d03 = [os.path.join(vehicle_path, f'{x}_saltpepper0d03') for x in directions]
    for p in paths_saltpepper0d03:
        mkdir_if_missing(p)
        
    paths_saltpepper0d04 = [os.path.join(vehicle_path, f'{x}_saltpepper0d04') for x in directions]
    for p in paths_saltpepper0d04:
        mkdir_if_missing(p)
        
    paths_saltpepper0d05 = [os.path.join(vehicle_path, f'{x}_saltpepper0d05') for x in directions]
    for p in paths_saltpepper0d05:
        mkdir_if_missing(p)
        
    paths_saltpepper0d06 = [os.path.join(vehicle_path, f'{x}_saltpepper0d06') for x in directions]
    for p in paths_saltpepper0d06:
        mkdir_if_missing(p)
        
    paths_saltpepper0d07 = [os.path.join(vehicle_path, f'{x}_saltpepper0d07') for x in directions]
    for p in paths_saltpepper0d07:
        mkdir_if_missing(p)
        
    paths_saltpepper0d08 = [os.path.join(vehicle_path, f'{x}_saltpepper0d08') for x in directions]
    for p in paths_saltpepper0d08:
        mkdir_if_missing(p)
        
    paths_saltpepper0d09 = [os.path.join(vehicle_path, f'{x}_saltpepper0d09') for x in directions]
    for p in paths_saltpepper0d09:
        mkdir_if_missing(p)
        
    paths_saltpepper0d10 = [os.path.join(vehicle_path, f'{x}_saltpepper0d10') for x in directions]
    for p in paths_saltpepper0d10:
        mkdir_if_missing(p)
                 
    paths_translatex25y0 = [os.path.join(vehicle_path, f'{x}_translatex25y0') for x in directions]
    for p in paths_translatex25y0:
        mkdir_if_missing(p)
                 
    paths_translatex0y25 = [os.path.join(vehicle_path, f'{x}_translatex0y25') for x in directions]
    for p in paths_translatex0y25:
        mkdir_if_missing(p)

    paths_translatex0ym25 = [os.path.join(vehicle_path, f'{x}_translatex0ym25') for x in directions]
    for p in paths_translatex0ym25:
        mkdir_if_missing(p)

    paths_translatexm25y0 = [os.path.join(vehicle_path, f'{x}_translatexm25y0') for x in directions]
    for p in paths_translatexm25y0:
        mkdir_if_missing(p)

    paths_gaussian = [os.path.join(vehicle_path, f'{x}_gaussian') for x in directions]
    for p in paths_gaussian:
        mkdir_if_missing(p)
        
    for p in paths_rot270:
        original_path = p.replace('_rot270', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                img = cv2.imread(os.path.join(original_path, image_name))
                rotated = rotate_bound(img, 270)
                cv2.imwrite(dst, rotated)
            except Exception as e:
                print(e)
                
    for p in paths_rot90:
        original_path = p.replace('_rot90', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                img = cv2.imread(os.path.join(original_path, image_name))
                rotated = rotate_bound(img, 90)
                cv2.imwrite(dst, rotated)
            except Exception as e:
                print(e)
                
    for p in paths_vflip:
        original_path = p.replace('_vflip', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                img = cv2.imread(os.path.join(original_path, image_name))
                vertical_img = img.copy()
                vertical_img = cv2.flip(img, 0)
                cv2.imwrite(dst, vertical_img)
            except Exception as e:
                print(e)
                
    for p in paths_gaussian:
        original_path = p.replace('_gaussian', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_gaussian(cv2.imread(image_path), 0, 0.6))
            except Exception as e:
                print(e)
                
    for p in paths_translatexm25y0:
        original_path = p.replace('_translatexm25y0', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, translate_image(
                    cv2.imread(image_path), -25, 0))
            except Exception as e:
                print(e)
                
    for p in paths_translatex0ym25:
        original_path = p.replace('_translatex0ym25', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, translate_image(
                    cv2.imread(image_path), 0, -25))
            except Exception as e:
                print(e)
                
    for p in paths_translatex0y25:
        original_path = p.replace('_translatex0y25', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, translate_image(
                    cv2.imread(image_path), 0, 25))
            except Exception as e:
                print(e)
                
    for p in paths_translatex25y0:
        original_path = p.replace('_translatex25y0', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, translate_image(
                    cv2.imread(image_path), 25, 0))
            except Exception as e:
                print(e)

    for p in paths_saltpepper0d01:
        original_path = p.replace('_saltpepper0d01', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.01))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d02:
        original_path = p.replace('_saltpepper0d02', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.02))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d03:
        original_path = p.replace('_saltpepper0d03', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.03))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d04:
        original_path = p.replace('_saltpepper0d04', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.04))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d05:
        original_path = p.replace('_saltpepper0d05', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.05))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d06:
        original_path = p.replace('_saltpepper0d06', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.06))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d07:
        original_path = p.replace('_saltpepper0d07', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.07))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d08:
        original_path = p.replace('_saltpepper0d08', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.08))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d09:
        original_path = p.replace('_saltpepper0d09', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.09))
            except Exception as e:
                print(e)
                
    for p in paths_saltpepper0d10:
        original_path = p.replace('_saltpepper0d10', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                cv2.imwrite(dst, add_noise_saltpepper(
                    cv2.imread(image_path), 0.10))
            except Exception as e:
                print(e)
                
    for p in paths_smartcrop:
        original_path = p.replace('_smartcrop', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                image_path = os.path.join(original_path, image_name)
                img = cv2.imread(image_path)
                (h, w) = img.shape[:2]
                image = Image.open(image_path)
                min_dimension = min(h, w)
                result = sc.crop(image, min_dimension, min_dimension)
                crops = result['crops']
                last_crop = crops[len(crops)//2]
                cropped = img[
                    last_crop['y']:last_crop['y']+last_crop['height'],
                    last_crop['x']:last_crop['x']+last_crop['width'],
                ]
                cv2.imwrite(dst, cropped)
            except Exception as e:
                print(e)
                
    for p in paths_hflip:
        original_path = p.replace('_hflip', '')
        image_names = os.listdir(original_path)
        random.shuffle(image_names)
        for image_name in image_names:
            try:
                dst = os.path.join(p, image_name)
                if os.path.exists(dst):
                    continue
                img = cv2.imread(os.path.join(original_path, image_name))
                horizontal_img = img.copy()
                horizontal_img = cv2.flip(img, 1)
                cv2.imwrite(dst, horizontal_img)
            except Exception as e:
                print(e)
            

현대_그랜저_그랜저 HG(11~16년)


/Users/jo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:38: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/Users/jo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


기아_K5_K5 2세대(15~18년)


KeyboardInterrupt: 